In [1]:
# !pip install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import logging
import multiprocessing
from datetime import datetime, timedelta

from src import modules as f
%load_ext autoreload
%autoreload 2

In [27]:
# Download, load, extract, model...
interval = '1d'
stock_set = f.download_interval_process(interval, 1)
print(f'Stock set: \n{stock_set}\n')

Stock set: 
{'MA', 'VZ', 'XEL', 'GS', 'AJG', 'MRK', 'AMZN', '^VIX', 'GOOGL', 'NRG', 'TTWO', 'KO', 'SRE', 'IBM', 'CVX', 'COST', 'WSM', 'WMT', 'CEG', 'DUK', 'D', 'PSX', 'ATO', 'NFLX', 'TMUS', 'SLB', 'RCL', 'UNH', 'FWONK', 'DASH', 'PGR', 'ABT', 'GOOG', 'PG', 'EOG', 'EXC', 'NEE', 'LLY', 'TDG', 'SO', 'ABBV', 'VST', 'MDLZ', 'CMCSA', 'SW', 'BAC', 'ACN', 'WMB', 'XOM', 'BRK-B', 'APH', 'LNG', 'HD', 'DIS', 'ORCL', 'FCNCA', 'PEP', 'MSI', 'AAPL', 'CSCO', 'DECK', 'JNJ', 'T', 'ADBE', 'WELL', 'MO', 'JPM', 'TSLA', 'WFC', 'TMO', 'NVDA', 'CL', 'AVGO', 'EXE', 'PM', 'CRM', 'COP', 'KMI', 'TGT', 'TPR', 'ISRG', 'DKNG', 'AEP', 'META', 'MSFT', 'SPGI', 'AMGN', 'OKE', 'BSX', 'V', 'PLTR', 'AXP'}



In [28]:
f.make_table_features_process(stock_set, interval, 8)

KeyboardInterrupt: 

In [ ]:
f.make_master_table(stock_set, interval)

In [ ]:
f.xg_boost_model(interval)

In [8]:
f.model_prospect('nvda', '1d')


Model Prediction 1d NVDA:
Predicted Next 1d Movement: UP

Model Prediction Probabilities:
no_change (0): 0.3285
up (1): 0.3846
down (2): 0.2869

Last Entry 1d NVDA Datetime Used for Prediction:
NOTE: It's in or contains the full 1d time interval.
EST: 2025-01-15 12:00:00 AM EST-0500
PDT: 2025-01-14 09:00:00 PM PST-0800


In [20]:
df_val_nvda = f.model_validation('nvda', '5m')
df_val_nvda.head(5)

,slow_sma_signal,fast_sma_signal,stdev20,stdev10,stdev5,vix_stdev20,vix_stdev10,vix_stdev5,vol_stdev20,vol_stdev10,...,day_of_month,day_of_week,hour_of_day,candle_cluster,direction,prediction,no_change_prob,up_prob,down_prob,dir_pred_match
Datetime,,,,,,,,,,,,,,,,,,,,,
2025-03-28 15:40:00+00:00,0,1,0.280278,0.224108,0.250095,0.268137,0.218744,0.227442,1.437108e+06,355927.654629,...,28,4,11,2.0,no_change,down,0.194525,0.367897,0.437578,0
2025-03-28 15:45:00+00:00,1,1,0.260314,0.242879,0.186914,0.217972,0.233069,0.121532,1.320801e+06,360969.747138,...,28,4,11,2.0,down,down,0.199911,0.367338,0.432751,1
2025-03-28 15:50:00+00:00,1,1,0.237532,0.219319,0.210216,0.194936,0.200336,0.101637,9.178419e+05,358427.285145,...,28,4,11,0.0,down,down,0.253257,0.358921,0.387821,1
2025-03-28 15:55:00+00:00,1,1,0.249590,0.258123,0.333554,0.186593,0.177401,0.132401,8.785887e+05,351818.980154,...,28,4,11,0.0,up,down,0.253257,0.358921,0.387821,0
2025-03-28 16:00:00+00:00,1,0,0.246052,0.253481,0.308369,0.176549,0.175942,0.090719,7.921207e+05,186090.329962,...,28,4,12,0.0,up,down,0.309507,0.282629,0.407865,0


In [21]:
df_val_nvda.dir_pred_match.sum()

20

In [22]:
df_val_nvda.shape

(50, 34)

In [25]:
20/50

0.4